# 飞桨常规赛：PALM病理性近视预测6月第6名方案

**此方案来自大佬的方案，并在这基础上做了一些新的尝试。基线地址：**

https://aistudio.baidu.com/aistudio/projectdetail/2020177?channelType=0&channel=0

本人有幸获得第6名

![](https://ai-studio-static-online.cdn.bcebos.com/e409b08e076b4a16b1a5295ec8c33efbd24a6aefe630420d924ced26cbacc05b)



# **一、赛题介绍**

## 1. 赛题介绍
常规赛：PALM病理性近视预测由ISBI2019 PALM眼科挑战赛赛题再现，其中病理性近视预测的任务旨在对眼科图像进行判断，获得该眼为病理性近视的概率。

数据集由中山大学中山眼科中心提供800张带病理性近视分类标注的眼底彩照供选手训练模型，另提供400张带标注数据供平台进行模型测试。图像分辨率为1444×1444，或2124×2056。

评价指标为AUC (Area Under Curve)，即ROC (Receiver operating characteristic) 曲线与坐标轴形成的面积。

![](https://ai-studio-static-online.cdn.bcebos.com/64afb1df4595408088e547b616423f447e5761ae13224ca4be559b0c598d598b)

比赛链接: [常规赛：PALM病理性近视预测](https://aistudio.baidu.com/aistudio/competition/detail/85)

## **2.数据简介**
PALM病理性近视预测常规赛由中山大学中山眼科中心提供800张带病理性近视分类标注的眼底彩照供选手训练模型，另提供400张带标注数据供平台进行模型测试。

## **3. 训练数据集**

文件名称：Train Train文件夹里有一个fundus_image文件夹和一个Classification.xlsx文件。fundus_image文件夹中数据均为眼底彩照，分辨率为1444×1444，或2124×2056。命名形如N0001.jpg、H0001.jpg、P0001.jpg和V0001.jpg。Classification.xlsx文件中为各眼底图像是否属于病理性近视，属于为1，不属于为0。

## **4.测试数据集**
文件名称：PALM-Testing400-Images 文件夹里包含400张眼底彩照，命名形如T0001.jpg。

## **5.提交内容及格式**
分类结果应在一个名为“Classification_Results.csv”的CSV文件中提供，第一列对应测试眼底图像的文件名(包括扩展名“.jpg”)，对应title为FileName；第二列包含诊断为PM的患者图像的分类预测概率(值从0.0到1.0)，对应title为PM Risk。示例如下：

![](https://ai-studio-static-online.cdn.bcebos.com/942be68153dc4c7eb4c0ac8c5b3dc4f19e083ee0ec75479a87a7c385296fbdee)




# **二、个人思路**

对训练集进行8：2的划分，然后对图片进行resize1024x1024，1120x1120的尺度下进行训练，数据增强加上了垂直翻转以及随机旋转，以及对学习率lr进行了不同的训练，从而得到不同的模型权重再进行测试，测试完后再进行结果融合。

**新增点：**

增加垂直翻转以及随机旋转，不同尺度+不同学习率

**创新点：**

结果融合

# **三、前期准备**

## 1. 包准备
既然是分类任务，首先想到的是PaddleClas。但是我忽然想到了大佬们搞的ppim。因为医疗影像我认为有重要的关注的地方，和遥感图像不太类似，注意力应该能取得较好的效果。听闻大佬们的ppim复现了很新的注意力网络，而且和源代码相比效果很好，所以决定试试。

In [ ]:
! pip -q install ppim -i https://pypi.python.org/pypi

## 2. 数据准备
### 2.1解压数据集
这个没啥好写的。

In [ ]:
# ! unzip -oq  /home/aistudio/data/data93465/常规赛：PALM病理性近视预测.zip
# ! rm -rf __MACOSX
# ! mv 常规赛：PALM病理性近视预测 PLAM

### 2.2 配置数据集
- 因为数据中本身就有了这个图像名和标签，我们就不用生成数据列表了。直接继承io中的Dataset，用于读取数据。因为与开始说数据的大小有两种分辨率，而且贼大，但是又不敢放的太小损失太多细节，所以这里都放到了1120X1120（也做了1024x1024尺度的尝试，发现1120x1120效果更好）。
- 划分的比列为0.8，图像增强再原有的基础上增加了垂直翻转以及随机翻转。

In [ ]:
import os
import pandas as pd
import numpy as np
import paddle
import paddle.vision.transforms as T
from paddle.io import Dataset
from PIL import Image

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [ ]:
class PLAMDatas(Dataset):
    def __init__(self, data_path, class_xls, mode='train', transforms=None):
        super(PLAMDatas, self).__init__()
        self.data_path = data_path
        self.name_label = (pd.read_excel(class_xls)).values
        lens = len(self.name_label)
        if mode == 'train':
            self.name_label = self.name_label[:int(0.8*lens)]
        else:
            self.name_label = self.name_label[int(0.8*lens):]
        self.transforms = transforms
        
    def __getitem__(self, index):
        name, label = self.name_label[index]
        data_path = os.path.join(self.data_path, name)
        data = np.asarray(Image.open(data_path).convert('RGB'))
        if self.transforms is not None:
            data = self.transforms(data)
        data = data.astype('float32')
        label = np.array(int(label)).astype('int64')
        return data, label
        
    def __len__(self):
        return len(self.name_label)

# 配置数据增广
train_transforms = T.Compose([

    T.Resize((1120, 1120), interpolation='bicubic'),
    T.ColorJitter(0.1, 0.1, 0.1, 0.1),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomRotation(90),
   	T.ToTensor()
])

val_transforms = T.Compose([
    T.Resize((1120, 1120), interpolation='bicubic'),
    T.ToTensor()
])

# 配置数据集
train_dataset = PLAMDatas(data_path='PLAM/Train/fundus_image', class_xls='PLAM/Train/Classification.xlsx', mode='train', transforms=train_transforms)
val_dataset = PLAMDatas(data_path='PLAM/Train/fundus_image', class_xls='PLAM/Train/Classification.xlsx', mode='test', transforms=val_transforms)

这里也是输出测试一下，看看数据读取有没有什么问题。避免后面报一堆错不知道哪儿去找问题。

In [ ]:
print(len(train_dataset), len(val_dataset))
for img, lab in train_dataset:
    print(img.shape, lab)
    break
for img ,lab in val_dataset:
    print(img.shape, lab)
    break

640 160
[3, 1120, 1120] 0
[3, 1120, 1120] 0


# **四、开始训练**

## 1.模型准备
基础的DistilledVisionTransformer，自己说用1120的。但是这样就没有预训练的参数了，只能自己跑了。patch_size也改了一下，太小的话空间占用多，运算慢，而且不太好。summary看一下，总算没问题了。

In [ ]:
import paddle
import paddle.nn as nn
from ppim import DistilledVisionTransformer

# 模型定义
model = DistilledVisionTransformer(
    img_size=1120,
    patch_size=64,
    class_dim=2)
params = paddle.load('save_models5/final.pdparams')
model.set_state_dict(params)
paddle.summary(model, (1, 3, 1120, 1120))
model = paddle.Model(model)

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-8     [[1, 3, 1120, 1120]]   [1, 768, 17, 17]      9,437,952   
 PatchEmbed-4   [[1, 3, 1120, 1120]]    [1, 289, 768]            0       
  Dropout-112     [[1, 291, 768]]       [1, 291, 768]            0       
 LayerNorm-76     [[1, 291, 768]]       [1, 291, 768]          1,536     
  Linear-151      [[1, 291, 768]]       [1, 291, 2304]       1,769,472   
  Dropout-113   [[1, 12, 291, 291]]   [1, 12, 291, 291]          0       
  Linear-152      [[1, 291, 768]]       [1, 291, 768]         590,592    
  Dropout-114     [[1, 291, 768]]       [1, 291, 768]            0       
 Attention-37     [[1, 291, 768]]       [1, 291, 768]            0       
  Identity-37     [[1, 291, 768]]       [1, 291, 768]            0       
 LayerNorm-77     [[1, 291, 768]]       [1, 291, 768]          1,536     
  Linear-153      [[1, 291, 768]]   

## 2 .开始训练
这里进行两次不同优化器的训练，首先用Adam跑了100epochs，让模型更快的收敛，然后再用SGD跑50epochs进行降loss。

使用了不同的学习率配合不同的优化器进行训练

1. CosineAnnealingDecay + Adam + bs64
2. PolynomialDecay + SGD + ClipGradByGlobalNorm + bs8

注:checkpiont放在save_models*文件夹里面

In [16]:
# # 模型准备
# lr = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=0.00000001, T_max=int(2*(800*0.9)), verbose=False)
# opt = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters(), weight_decay=paddle.regularizer.L2Decay(1e-7))


lr = paddle.optimizer.lr.PolynomialDecay(learning_rate=3e-9, decay_steps=1000)
opt = paddle.optimizer.SGD(learning_rate=lr, parameters=model.parameters(), \
                           weight_decay=paddle.regularizer.L2Decay(1e-9), grad_clip=paddle.nn.ClipGradByGlobalNorm(clip_norm=1.0))
                           
# last
# opt_params = paddle.load('save_models/85.pdparams')
# opt.set_state_dict(opt_params)
loss = nn.CrossEntropyLoss()
metric = paddle.metric.Accuracy()
model.prepare(optimizer=opt, loss=loss, metrics=metric)
visualdl=paddle.callbacks.VisualDL(log_dir='visual_log')

# 模型微调
model.fit(
    train_data=train_dataset, 
    eval_data=val_dataset, 
    batch_size=32, 
    epochs=50,
    eval_freq=5,
    log_freq=1, 
    save_dir='save_models7', 
    save_freq=5,
    verbose=1, 
    drop_last=True, 
    shuffle=True,
    num_workers=0,
    callbacks=[visualdl]
)

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):


step 20/20 [==============================] - loss: 0.2596 - acc: 0.9406 - 5s/step        
save checkpoint at /home/aistudio/save_models7/0
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 5/5 [==============================] - loss: 0.1552 - acc: 0.9625 - 3s/step         
Eval samples: 160
Epoch 2/50
step 20/20 [==============================] - loss: 0.1823 - acc: 0.9484 - 5s/step        
Epoch 3/50
step 20/20 [==============================] - loss: 0.1013 - acc: 0.9484 - 5s/step        
Epoch 4/50
step 20/20 [==============================] - loss: 0.0713 - acc: 0.9453 - 5s/step        
Epoch 5/50
step 20/20 [==============================] - loss: 0.1338 - acc: 0.9594 - 5s/step        
Epoch 6/50
step 20/20 [==============================] - loss: 0.0655 - acc: 0.9484 - 5s/step        
save checkpoint at /home/aistudio/save_models7/5
Eval begin...
The loss value printed in the log is the current batch,

KeyboardInterrupt: 

# **五、预测结果**

## 4. 模型预测
- 结果必须升序排列

In [17]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import paddle.vision.transforms as T
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from ppim import DistilledVisionTransformer

save_path = '10.0.csv'
file_path = 'PLAM/PALM-Testing400-Images'
imgs_name = os.listdir(file_path)

model = DistilledVisionTransformer(
    img_size=1120,
    patch_size=64,
    class_dim=2)
params = paddle.load('save_models7/20.pdparams')
model.set_state_dict(params)
model.eval()

inf_transforms = T.Compose([
    # T.Resize((1024, 1024), interpolation='bicubic'),
    # T.ColorJitter(0.1, 0.1, 0.1, 0.1),
    # T.RandomHorizontalFlip(),
    # T.RandomRotation(90),
   	# T.ToTensor()
    T.Resize((1120, 1120), interpolation='bicubic'),  # 1120X1120
    T.ToTensor()
])

pre_data = []
for img_name in imgs_name:
    data_path = os.path.join(file_path, img_name)
    data = np.asarray(Image.open(data_path).convert('RGB'))
    data = inf_transforms(data)
    data = data.astype('float32').reshape([1, 3, 1120, 1120])
    pre = model(data)
    pre = F.softmax(pre)
    print([img_name, pre.numpy()[0][1]])
    pre_data.append([img_name, pre.numpy()[0][1]])


['T0103.jpg', 0.012307137]
['T0210.jpg', 0.90871423]
['T0323.jpg', 0.014455602]
['T0134.jpg', 0.9990803]
['T0066.jpg', 0.9962853]
['T0001.jpg', 0.99403954]
['T0225.jpg', 0.45718023]
['T0286.jpg', 0.041713268]
['T0209.jpg', 0.99929214]
['T0233.jpg', 0.03114533]
['T0366.jpg', 0.059233658]
['T0202.jpg', 0.0029475705]
['T0278.jpg', 0.99712855]
['T0398.jpg', 0.9990043]
['T0078.jpg', 0.0048059905]
['T0270.jpg', 0.9877489]
['T0289.jpg', 0.81314903]
['T0232.jpg', 0.8559369]
['T0253.jpg', 0.1409466]
['T0330.jpg', 0.0064832186]
['T0043.jpg', 0.6406758]
['T0370.jpg', 0.9222939]
['T0172.jpg', 0.99530596]
['T0061.jpg', 0.86990035]
['T0167.jpg', 0.9949851]
['T0349.jpg', 0.94388616]
['T0360.jpg', 0.9947523]
['T0333.jpg', 0.0065847537]
['T0297.jpg', 0.20527397]
['T0053.jpg', 0.012088187]
['T0228.jpg', 0.005353403]
['T0127.jpg', 0.9989631]
['T0381.jpg', 0.003037737]
['T0344.jpg', 0.9908296]
['T0352.jpg', 0.9854972]
['T0189.jpg', 0.012986695]
['T0100.jpg', 0.49359265]
['T0341.jpg', 0.9996692]
['T0075.jp

In [18]:
new_data = sorted(pre_data)

print(new_data)

[['T0001.jpg', 0.99403954], ['T0002.jpg', 0.008706468], ['T0003.jpg', 0.0043857438], ['T0004.jpg', 0.9052263], ['T0005.jpg', 0.99958795], ['T0006.jpg', 0.032943066], ['T0007.jpg', 0.9691701], ['T0008.jpg', 0.9956103], ['T0009.jpg', 0.9777526], ['T0010.jpg', 0.9446918], ['T0011.jpg', 0.0107444525], ['T0012.jpg', 0.9995516], ['T0013.jpg', 0.9843315], ['T0014.jpg', 0.6884365], ['T0015.jpg', 0.98728323], ['T0016.jpg', 0.00441275], ['T0017.jpg', 0.0026218523], ['T0018.jpg', 0.9953727], ['T0019.jpg', 0.0037111046], ['T0020.jpg', 0.9926979], ['T0021.jpg', 0.0043539344], ['T0022.jpg', 0.004218383], ['T0023.jpg', 0.99892056], ['T0024.jpg', 0.01631227], ['T0025.jpg', 0.0060505695], ['T0026.jpg', 0.9908313], ['T0027.jpg', 0.022977568], ['T0028.jpg', 0.0064003114], ['T0029.jpg', 0.98273057], ['T0030.jpg', 0.5582529], ['T0031.jpg', 0.032938343], ['T0032.jpg', 0.012693948], ['T0033.jpg', 0.0071240864], ['T0034.jpg', 0.9992411], ['T0035.jpg', 0.99029607], ['T0036.jpg', 0.9984432], ['T0037.jpg', 0.991

**生成结果文件**

In [19]:
df = pd.DataFrame(new_data, columns=['FileName', 'PM Risk'])
df.to_csv(save_path, index=None)

# **六、结果分析**

近视预测：

lr=3e-6（原始学习率）

1024x1024：

Adam-100-0.9596

Adam-100+SGD-50-0.9540

1120x1120：

Adam-50-0.97442(预测1120)

Adam-50-0.97514(预测1024)

发现1120x1120准确率都比较高，因此用1120x1120尺寸，用不同学习率lr，以及不同优化器进行训练

lr=0.0000125

Adam-100：

1120x1120,+垂直反转:0.99132(预测1024)

1120x1120,+垂直反转:**0.99202**(预测1120)

Adam-100+SGD-60:0.99152

Adam-100：

1120x1120，再+transforms.RandomRotation(90),#随机旋转，：**0.99204**

Adam-100+sgd-30：

1120x1120，再+transforms.RandomRotation(90),#随机旋转，：**0.99214**

结果融合：**0.99232**

# **七、创新点：结果融合**（能够提一点分）

**注意事项：**

进行结果融合的模型，模型精度理应相近不能差别太大，否则融合模型精度会低于最优单一模型精度

In [21]:
import numpy as np
import pandas as pd
df1=pd.read_csv('6.0.csv')
df2=pd.read_csv('9.0+sgd.csv')
# df3=pd.read_csv('10.0.csv')
dfs=[df1,df2]
PM_Risk=[]
File_Name=[]

for i in range(len(df1)):
    File_Name.append(dfs[0]['FileName'][i])
    avg=(sum(np.array(dfs[x]['PM Risk'][i]) for x in range(len(dfs))))/len(dfs)
  
    
    PM_Risk.append(avg)

submission = pd.DataFrame(data={
                            "FileName": File_Name,
                            "PM Risk": PM_Risk
                        })
submission=submission.sort_values(by='FileName')
submission.to_csv("resultx2.csv", index=False)

# 八、分析总结

1.提高图片尺寸，相应的可以提高预测精度

2.传统的数据增强是增分的稳定点，需要多加尝试

3.可以进一步尝试不同的优化器和学习率衰减函数结合





## **病理性近视预测相关论文**

1、 [PPIM](https://github.com/AgentMaker/Paddle-Image-Models)

2、https://www.sciencedirect.com/science/article/pii/S0169260720317533

3、https://ieeexplore.ieee.org/abstract/document/8929252/